# **Edge Detection**

In [8]:
import os
import cv2
import numpy as np

img = cv2.imread(os.path.join('..', 'assets', 'footballPlayer.jpg'))

img_edge = cv2.Canny(img, 100, 200) # 100 is the minVal and 200 is the maxVal
cv2.imshow('Original Image', img)
cv2.imshow('Edge Detected Image', img_edge)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# dilate : means to expand the white region in the image or to increase the size of foreground objects.

img_dilated = cv2.dilate(img_edge, None, iterations=1) # None means using the default 3x3 kernel

# img_dilated = cv2.dilate(img_edge, np.ones((5,5), dtype=np.int8)) # make it 3*3 will make it less thicker

cv2.imshow('Original edged Image', img_edge)  
cv2.imshow('Dilated Image', img_dilated)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
# erode : means to erode away the boundaries of foreground object or to decrease the size of foreground objects (opposite to dilate).
img_eroded = cv2.erode(img_edge, np.ones((2,2), dtype=np.int8)) # None means using the default 3x3 kernel
cv2.imshow('Original edged Image', img_edge)
cv2.imshow('Eroded Image', img_eroded)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### **Refrences:**
https://opencv.org/blog/edge-detection-using-opencv/

---

# **Notes**

## **Core Algorithm: Canny Edge Detector**

* **Function:** The lesson focuses on the **Canny Edge Detector**, which is implemented using `cv2.Canny()`.
* **Alternative Methods:** The instructor briefly mentions other operators like **Sobel** and **Laplacian** but focuses on Canny for this lesson.
* **Syntax:** `cv2.Canny(image, threshold1, threshold2)`.
* **Parameters (Hysteresis Thresholding):**
    * The function requires two integer arguments (e.g., 100 and 200).
    * These values represent the **min** and **max** thresholds for the hysteresis process.
    * **Tuning:** The instructor recommends using **trial and error** to find the best numbers. The algorithm is robust, so a range of values (e.g., 100-200 or 50-200) often produces good results.



## **Morphological Transformations**

#### The instructor introduces two additional functions often used to improve the visualization of edges:

* **Dilation (`cv2.dilate`):**
    * **Effect:** It makes the white regions (the detected edges) **thicker**.
    * **Syntax:** `cv2.dilate(image, kernel, iterations=1)`.
    * **Kernel:** Requires a NumPy array (e.g., a 5x5 matrix of ones) to define how much to thicken the lines.
    * **Use Case:** Useful for making faint edges more visible or connecting broken lines.


* **Erosion (`cv2.erode`):**
    * **Effect:** It makes the white regions **thinner**.
    * **Syntax:** `cv2.erode(image, kernel, iterations=1)`.
    * **Relationship:** It performs the opposite operation of dilation.
    
---

---

## **Algorithms:**

### **1. Canny Edge Detection Algorithm**

The Canny algorithm is a multi-stage process designed to be an "optimal" edge detector. It doesn't just find changes in color; it cleans them up to produce thin, continuous lines.

1. **Noise Reduction (Gaussian Filter):**
* Since edge detection is very sensitive to noise (random speckles), the algorithm first smoothes the image using a Gaussian blur kernel (like a weighted average) to remove "false" edges caused by noise.


2. **Gradient Calculation:**
* It calculates the intensity gradient of the image (typically using Sobel kernels).
* It determines both the **magnitude** (how strong the edge is) and the **direction** (where the edge is pointing) for every pixel.


3. **Non-Maximum Suppression (Edge Thinning):**
* The algorithm scans every pixel along the gradient direction.
* If a pixel is **not** the "peak" (maximum value) compared to its neighbors in the direction of the edge, it is set to zero (suppressed).
* *Result:* This converts thick, blurry edges into thin, sharp lines (1 pixel wide).


4. **Double Thresholding:**
* This separates pixels into three categories based on the two threshold values you provided (e.g., 100 and 200):
* **Strong Edge:** Intensity > High Threshold (Definitely an edge).
* **Weak Edge:** Intensity is between Low and High Threshold (Maybe an edge, maybe noise).
* **Non-Edge:** Intensity < Low Threshold (Discarded).




5. **Edge Tracking by Hysteresis:**
* The algorithm decides what to do with the "Weak Edges."
* If a Weak Edge pixel is connected to a Strong Edge pixel, it is preserved.
* If a Weak Edge pixel is isolated (not connected to a strong one), it is discarded as noise.



---

### **2. Sobel Operator Algorithm**

The Sobel operator is a discrete differentiation operator. It computes an approximation of the gradient of the image intensity function.

1. **Convolution (X and Y):**
* The algorithm slides two  kernels (matrices) over the image:
* **Kernel **: Calculates the difference in intensity horizontally (detects vertical edges).
* **Kernel **: Calculates the difference in intensity vertically (detects horizontal edges).




2. **Gradient Magnitude Calculation:**
* For each pixel, it combines the X and Y results to find the total edge strength using the Pythagorean theorem:




3. **Gradient Direction Calculation (Optional):**
* It calculates the angle of the edge (perpendicular to the gradient):





---

### **3. Laplacian Algorithm**

Unlike Sobel (which uses the first derivative/slope), Laplacian uses the **second derivative** (how fast the slope is changing). It is isotropic (rotation invariant), meaning it detects edges equally well in all directions.

1. **Calculate Second Derivative:**
* The algorithm calculates the sum of the second derivatives in the x and y directions.
* Mathematically, it looks for the "curvature" of the intensity graph.


2. **Zero-Crossing Detection:**
* In a first derivative (Sobel), an edge is a peak (maximum value).
* In a second derivative (Laplacian), an edge is where the values cross from positive to negative (a **zero-crossing**).
* The algorithm identifies edges by locating these zero-crossing points.

---

---
# **Coding**

### **1. Canny Edge Detection**

This is the method recommended in the lesson for its robustness.

* **Step 1:** Read the image.
* **Step 2:** Call `cv2.Canny(image, threshold1, threshold2)`.
* **Step 3:** Tune the two thresholds (hysteresis). The instructor suggests trial and error, often starting with values like 100 and 200.

```python
import cv2

img = cv2.imread('image.jpg')
# Canny handles internal blurring, but reducing noise externally is often good practice
edges = cv2.Canny(img, 100, 200)

cv2.imshow('Canny', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

These are the **Hysteresis Thresholds**:

* **100 (MinVal):** Any gradient value **below** this is strictly discarded (considered not an edge).
* **200 (MaxVal):** Any gradient value **above** this is strictly accepted (considered a "sure" edge).
* **Between 100 and 200:** These are "weak" edges. They are only kept if they are connected to a "sure" edge; otherwise, they are discarded.

```

---

### **2. Sobel Operator**

The Sobel operator calculates the gradient (change in intensity) separately for the horizontal (x) and vertical (y) directions.

* **Step 1:** Read the image and convert to **Grayscale**.
* **Step 2:** Apply a Gaussian Blur to remove noise (Sobel is sensitive to noise).
* **Step 3:** Calculate gradients in X and Y direction using `cv2.Sobel`.
* **Step 4:** Combine the two gradients.

```python
import cv2

img = cv2.imread('image.jpg', cv2.IMREAD_GRAYSCALE) # cv2.IMREAD_GRAYSCALE -> load the image directly as a single-channel grayscale image, ignoring any color information.
img_blur = cv2.GaussianBlur(img, (3, 3), 0)

# Sobel X: Vertical edges (dx=1, dy=0)
sobelx = cv2.Sobel(img_blur, cv2.CV_64F, 1, 0, ksize=5)

# Sobel Y: Horizontal edges (dx=0, dy=1)
sobely = cv2.Sobel(img_blur, cv2.CV_64F, 0, 1, ksize=5)

cv2.imshow('Sobel X', sobelx)
cv2.imshow('Sobel Y', sobely)
cv2.waitKey(0)
cv2.destroyAllWindows()

Here is the very brief breakdown:

* **`dx`**: The order of the derivative in **x**. Setting `dx=1` tells OpenCV to calculate the change in pixel intensity horizontally (finding vertical edges).
* **`dy`**: The order of the derivative in **y**. Setting `dy=1` tells OpenCV to calculate the change in pixel intensity vertically (finding horizontal edges).
* **`ksize`**: The **Kernel Size**. It defines the width and height of the square window (matrix) used to calculate the derivative (e.g., `5` means a  window).

```

---

### **3. Laplacian Edge Detector**

The Laplacian calculates the second derivative of the image intensity. It detects edges where the intensity changes rapidly in *any* direction.

* **Step 1:** Read the image and convert to **Grayscale**.
* **Step 2:** Apply a Gaussian Blur (crucial, as Laplacian is very sensitive to noise).
* **Step 3:** Apply `cv2.Laplacian`.

```python
import cv2

img = cv2.imread('image.jpg', cv2.IMREAD_GRAYSCALE)
img_blur = cv2.GaussianBlur(img, (3, 3), 0) # (3,3) -> kernal size, 0 -> st deviation.

# The second argument (cv2.CV_64F) allows for negative gradients (black-to-white transitions)
laplacian = cv2.Laplacian(img_blur, cv2.CV_64F)

cv2.imshow('Laplacian', laplacian)
cv2.waitKey(0)
cv2.destroyAllWindows()

```

### **Why we use cv2.CV_64F (64-bit float) instead of the standard 8-bit integer for Sobel and Laplacian?** 

We use `cv2.CV_64F` because **edge detection involves negative numbers**, which 8-bit images cannot store.

* **The Issue:** Standard images are `uint8` (0 to 255). They cannot hold negative values.
* **The Math:** An edge is a change in brightness.
* **Dark  Bright:** Positive change (e.g., ).
* **Bright  Dark:** Negative change (e.g., ).


* **The Result:** If you use `uint8`, all negative edges (bright-to-dark transitions) are clamped to **0** (black) and lost.
* **The Fix:** We use `cv2.CV_64F` (floats) to keep the negative numbers, then take the absolute value later to view the edges correctly.